# Data Wrangling

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

data_csv = 'data/scraped_data.csv'

df = pd.read_csv(data_csv)
raw_df = df # in case this is needed for lookups

## Cleaning the data
In this section, we remove duplicates as well as columns that are not useful for price prediction, fill empty values, and map column values to numbers.

In [62]:
df['price'].describe()

count    1379.000000
mean     2083.573604
std       851.479067
min       430.000000
25%      1500.000000
50%      1800.000000
75%      2500.000000
max      7000.000000
Name: price, dtype: float64

In [68]:
df = raw_df
# Remove duplidate rows
df = df.drop_duplicates()

# Drop some not very useful columns;
# These are columns that always have the same value (e.g. county will always be Dublin), 
# or columns that don't seem to correlate with the price at all
df = df.drop(['property_id','environment','page_name','platform',
             'property_category','seller_name','published_date','facility',
             'property_type','lease_units','county','currency','longitude','latitude'],1) 

# Get dictionary of unique areas
areas = set(df['area'])
areas_dict = {x:float(i) for i,x in enumerate(areas)}
df['area'] = df['area'].map(areas_dict)

# Remove superexpensive exceptions
df = df.loc[df['price'] < 4300]
df = df.loc[df['price'] >= 500]

# Replace 'seller_type' values with numbers
df['seller_type'] = df['seller_type'].map({'agent': 1.0, 'private': 0.0})

# Replace 'furnished' values with numbers
df['furnished'] = df['furnished'].map({'yes': 1.0, 'no': 0.0, 'either': 3.0})

# Replace 'open_viewing' values with numbers
df['open_viewing'] = df['open_viewing'].map({'yes': 1.0, 'no': 0.0})

# Convert int values to floats
df['no_of_photos'] = df['no_of_photos'].astype(float)
df['available_for'] = df['available_for'].astype(float)
df['price'] = df['price'].astype(float)
df['beds'] = df['beds'].astype(float)
df['bathrooms'] = df['bathrooms'].astype(float)

# Replace True values with 1, False and NaN with 0
df = df.replace({True:1.0}).replace({False:0.0}).fillna(0.0)

# Adding pricerange column
df['pricerange'] = 4
df.loc[df['price'] < 1000, 'pricerange'] = 0
df.loc[(df['price'] > 999) & (df['price'] < 2000), 'pricerange'] = 1
df.loc[(df['price'] > 1999) & (df['price'] < 3000), 'pricerange'] = 2
df.loc[(df['price'] > 2999) & (df['price'] < 4000), 'pricerange'] = 3
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1348 entries, 0 to 1378
Data columns (total 29 columns):
property_title              1348 non-null object
seller_id                   1348 non-null float64
seller_type                 1348 non-null float64
open_viewing                1348 non-null float64
no_of_photos                1348 non-null float64
available_from              1348 non-null object
available_for               1348 non-null float64
area                        1348 non-null float64
furnished                   1348 non-null float64
bathrooms                   1348 non-null float64
beds                        1348 non-null float64
price_frequency             1348 non-null object
price                       1348 non-null float64
Parking                     1348 non-null float64
Cable Television            1348 non-null float64
Dryer                       1348 non-null float64
Garden / Patio / Balcony    1348 non-null float64
Washing Machine             1348 non-null floa

,property_title,seller_id,seller_type,open_viewing,no_of_photos,available_from,available_for,area,furnished,bathrooms,...,Pets Allowed,Wheelchair Access,Central Heating,Microwave,Smoking,Dishwasher,House Alarm,Internet,property_description,pricerange
0,"1 BED, Tallaght Cross West, Tallaght, Dublin 24",9871.0,1.0,0.0,3.0,2017-10-01,12.0,66.0,0.0,1.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,*** UNFURNISHED*** Irish Residential Propert...,1
1,"1 Palace Street, Dublin 2, Dublin 2",7549.0,1.0,0.0,6.0,2017-09-15,12.0,65.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,***** HERBERT PROPERTY SERVICES****are delight...,2
2,"10 Clarinda House, Clarinda Park West, Dun Lao...",0.0,0.0,0.0,8.0,2017-09-18,6.0,9.0,1.0,1.0,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,Rating to be updated (new windows to be delive...,1
3,"10 Gandon Hall, Gardners Street, Dublin 1, Du...",0.0,0.0,0.0,5.0,2017-10-06,0.0,72.0,1.0,1.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,SHORT TERM OR LONG TERM LETTING-A renovated on...,1
4,"10 The Clayton, The Gasworks, Barrow Street, D...",0.0,0.0,0.0,7.0,2017-11-10,12.0,23.0,1.0,2.0,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,"CLAYTON, THE GASWORKS, BARROW STREET, DUBLIN 4...",2


In [69]:
print ("Dimension of train data {}".format(df.shape))

Dimension of train data (1348, 29)


In [70]:
print(df.columns.values)

['property_title' 'seller_id' 'seller_type' 'open_viewing' 'no_of_photos'
 'available_from' 'available_for' 'area' 'furnished' 'bathrooms' 'beds'
 'price_frequency' 'price' 'Parking' 'Cable Television' 'Dryer'
 'Garden / Patio / Balcony' 'Washing Machine' 'Serviced Property'
 'Pets Allowed' 'Wheelchair Access' 'Central Heating' 'Microwave' 'Smoking'
 'Dishwasher' 'House Alarm' 'Internet' 'property_description' 'pricerange']


## Splitting the data into weekly and monthly rentals

In [71]:
df_weekly_rates = df.loc[df['price_frequency'] == 'weekly']
df_weekly_rates = df_weekly_rates.drop('price_frequency', 1)
print(df_weekly_rates.shape)

df_monthly_rates = df.loc[df['price_frequency'] == 'monthly']
df_monthly_rates = df_monthly_rates.drop('price_frequency', 1)

# There seems to be an ad with 47 beds for 750/months. removing this
df_monthly_rates = df_monthly_rates[df_monthly_rates.beds < 20]
print(df_monthly_rates.shape)

(183, 28)
(1164, 28)


## Save data as CSVs

In [72]:
import json
df_monthly_rates.to_csv('data/cleaned_monthly.csv')
json.dump(areas_dict, open('data/areas_dict.csv','w'))